In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('IAG.MC.csv')

df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,1.8495,1.8500,1.6550,1.6610,1.6610,38696800
1,2021-01-05,1.6420,1.6995,1.6000,1.6575,1.6575,24855270
2,2021-01-06,1.6830,1.7600,1.6465,1.7505,1.7505,21294975
3,2021-01-07,1.7575,1.7700,1.6760,1.7375,1.7375,20918102
4,2021-01-08,1.7640,1.7980,1.7275,1.7345,1.7345,33522834
...,...,...,...,...,...,...,...
809,2024-02-28,1.8255,1.8310,1.7840,1.7860,1.7860,8953255
810,2024-02-29,1.8210,1.8340,1.7260,1.7270,1.7270,27594692
811,2024-03-01,1.7450,1.7595,1.7140,1.7140,1.7140,12356338
812,2024-03-04,1.7105,1.7230,1.6860,1.6920,1.6920,14598128


In [4]:
df.columns = df.columns.str.lower()

df = df [['date','open','high','low','close','adj close']]

df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")

df.sort_values(by='date',ascending= True, inplace=True)

fecha_inicio = '2021-01-01'
fecha_fin = '2024-03-01'

name = pd.DataFrame(df[(df['date'] >= fecha_inicio) & (df['date'] <= fecha_fin)]).reset_index(drop=True)

C:\Users\javie\AppData\Local\Temp\ipykernel_18348\1077789613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
C:\Users\javie\AppData\Local\Temp\ipykernel_18348\1077789613.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='date',ascending= True, inplace=True)


In [5]:
df

,date,open,high,low,close,adj close
0,2021-01-04,1.8495,1.8500,1.6550,1.6610,1.6610
1,2021-01-05,1.6420,1.6995,1.6000,1.6575,1.6575
2,2021-01-06,1.6830,1.7600,1.6465,1.7505,1.7505
3,2021-01-07,1.7575,1.7700,1.6760,1.7375,1.7375
4,2021-01-08,1.7640,1.7980,1.7275,1.7345,1.7345
...,...,...,...,...,...,...
809,2024-02-28,1.8255,1.8310,1.7840,1.7860,1.7860
810,2024-02-29,1.8210,1.8340,1.7260,1.7270,1.7270
811,2024-03-01,1.7450,1.7595,1.7140,1.7140,1.7140
812,2024-03-04,1.7105,1.7230,1.6860,1.6920,1.6920


In [6]:
def preparar_modelo(DF):


    DF_filt = DF[['date','close']]

    DF_filt.close = DF.close.round(2)

    DF_filt= DF_filt.reset_index(drop=True)

    new_df = pd.DataFrame(columns=['d-4', 'd-3', 'd-2', 'd-1', 'do'])

    for i in range(len(new_df.columns)):
        new_df[new_df.columns[i]] = DF_filt['close'].shift(-i)

    new_df['date']= DF_filt.date

    new_df.insert(0, 'date', new_df.pop('date'))

    new_df.dropna(axis=0,inplace=True)

    return new_df

In [7]:
new_df = preparar_modelo(df)

new_df

C:\Users\javie\AppData\Local\Temp\ipykernel_18348\3138495832.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filt.close = DF.close.round(2)


,date,d-4,d-3,d-2,d-1,do
0,2021-01-04,1.66,1.66,1.75,1.74,1.73
1,2021-01-05,1.66,1.75,1.74,1.73,1.70
2,2021-01-06,1.75,1.74,1.73,1.70,1.76
3,2021-01-07,1.74,1.73,1.70,1.76,1.71
4,2021-01-08,1.73,1.70,1.76,1.71,1.82
...,...,...,...,...,...,...
805,2024-02-22,1.80,1.78,1.81,1.83,1.79
806,2024-02-23,1.78,1.81,1.83,1.79,1.73
807,2024-02-26,1.81,1.83,1.79,1.73,1.71
808,2024-02-27,1.83,1.79,1.73,1.71,1.69


In [8]:
def modelo_train(new_df):

    # Importar herramientas:

    from sklearn.model_selection import train_test_split as tts 
    from sklearn.ensemble import RandomForestRegressor as RFR 
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP
    from sklearn.metrics import mean_squared_error as mse

    # Separar X e y:

    X = new_df.drop(['do','date'],axis=1)
    y = new_df.do

    # Ejecutar tts:

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # Iniciar Modelos

    rfr=RFR()
    omp= OMP()

    # Entrenar

    rfr.fit(X_train, y_train)
    omp.fit(X_train, y_train)
     
    # Predecir

    y_pred_rfr=rfr.predict(X_test)
    y_pred_omp=omp.predict(X_test)

    # Mostrar rmse de cada modelo   
   
    return 'RMSE RFR ======',mse(y_test, y_pred_rfr, squared=False),'REMS OMP ======',mse(y_test, y_pred_omp, squared=False)

In [9]:
modelo_train(new_df)

('RMSE RFR ======',
 0.04878899834736688,
 'REMS OMP ======',
 0.04484229194477522)

In [10]:
def model(new_df):

    fecha_inicio = '2024-01-01'
    fecha_fin = '2024-02-26'

    # Filtrar el DataFrame
    mask = (new_df['date'] >= fecha_inicio) & (new_df['date'] <= fecha_fin)
    df_pred = new_df.loc[mask]

    df_pred_1 = df_pred[['d-4','d-3','d-2','d-1']]
    
    
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP

    omp= OMP()

    predicciones_y = []

    # Iterar sobre las 40 posiciones
    for i in range(40):
        # Seleccionar la porción correspondiente de X e y para esta iteración
        X_iteracion = new_df.drop(['date', 'do'], axis=1)[:-40+i]
        y_iteracion = new_df.do[:-40+i]

        # Ajustar el modelo
        omp.fit(X_iteracion, y_iteracion)

        # Seleccionar la fila correspondiente de df_pred para la predicción
        pred = df_pred_1[i:i+1]

        # Realizar la predicción y almacenarla en la lista
        y_pred_omp = omp.predict(pred)
        predicciones_y.append(y_pred_omp[0])   

    pred = pd.DataFrame(predicciones_y)

    pred.columns = ['prediccion']

    df_pred = df_pred.reset_index(drop=True)
    
    df_pred['pred'] = pred.prediccion.round(2)

    df_pred['error'] = df_pred.pred.round(2) - df_pred.do  

    return  df_pred

In [11]:
df_pred = model(new_df)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error
0,2024-01-02,1.79,1.74,1.78,1.77,1.77,1.77,0.00
1,2024-01-03,1.74,1.78,1.77,1.77,1.76,1.77,0.01
2,2024-01-04,1.78,1.77,1.77,1.76,1.75,1.76,0.01
3,2024-01-05,1.77,1.77,1.76,1.75,1.73,1.75,0.02
4,2024-01-08,1.77,1.76,1.75,1.73,1.68,1.73,0.05
5,2024-01-09,1.76,1.75,1.73,1.68,1.69,1.68,-0.01
6,2024-01-10,1.75,1.73,1.68,1.69,1.66,1.69,0.03
7,2024-01-11,1.73,1.68,1.69,1.66,1.66,1.66,0.00
8,2024-01-12,1.68,1.69,1.66,1.66,1.68,1.66,-0.02
9,2024-01-15,1.69,1.66,1.66,1.68,1.68,1.68,0.00


In [12]:
def escenario(x):

    import math

    df_pred['stock'] = (x/df_pred.do).apply(math.floor)

    df_pred['day_gap'] = df_pred.do.diff()

    df_pred['money'] = df_pred.stock*df_pred.day_gap

    df_pred['evolution'] = x + df_pred['money'].iloc[1:].cumsum().round(2)

    max_money_index = df_pred['money'].idxmax()
    
    df_pred['opti_real'] = x + df_pred['money'].iloc[max_money_index:].cumsum().round(2)

    min_money_index = df_pred['pred'].idxmin()
    
    df_pred['opti_pred'] = x + df_pred['money'].iloc[min_money_index:].cumsum().round(2)

    return df_pred

In [13]:
df_pred = escenario(100)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,2024-01-02,1.79,1.74,1.78,1.77,1.77,1.77,0.00,56,NaN,NaN,NaN,NaN,NaN
1,2024-01-03,1.74,1.78,1.77,1.77,1.76,1.77,0.01,56,-0.01,-0.56,99.44,NaN,NaN
2,2024-01-04,1.78,1.77,1.77,1.76,1.75,1.76,0.01,57,-0.01,-0.57,98.87,NaN,NaN
3,2024-01-05,1.77,1.77,1.76,1.75,1.73,1.75,0.02,57,-0.02,-1.14,97.73,NaN,NaN
4,2024-01-08,1.77,1.76,1.75,1.73,1.68,1.73,0.05,59,-0.05,-2.95,94.78,NaN,NaN
5,2024-01-09,1.76,1.75,1.73,1.68,1.69,1.68,-0.01,59,0.01,0.59,95.37,NaN,NaN
6,2024-01-10,1.75,1.73,1.68,1.69,1.66,1.69,0.03,60,-0.03,-1.80,93.57,NaN,NaN
7,2024-01-11,1.73,1.68,1.69,1.66,1.66,1.66,0.00,60,0.00,0.00,93.57,NaN,100.00
8,2024-01-12,1.68,1.69,1.66,1.66,1.68,1.66,-0.02,59,0.02,1.18,94.75,NaN,101.18
9,2024-01-15,1.69,1.66,1.66,1.68,1.68,1.68,0.00,59,0.00,0.00,94.75,NaN,101.18


In [14]:
def dafatrame(df_pred,name):
    

    df_pred['name'] = name
    
    df_pred = df_pred[['name','date','do','pred','error','stock','day_gap','money','evolution','opti_real','opti_pred']]

    return df_pred

In [15]:
df= dafatrame(df_pred,'iag')

df

,name,date,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,iag,2024-01-02,1.77,1.77,0.00,56,NaN,NaN,NaN,NaN,NaN
1,iag,2024-01-03,1.76,1.77,0.01,56,-0.01,-0.56,99.44,NaN,NaN
2,iag,2024-01-04,1.75,1.76,0.01,57,-0.01,-0.57,98.87,NaN,NaN
3,iag,2024-01-05,1.73,1.75,0.02,57,-0.02,-1.14,97.73,NaN,NaN
4,iag,2024-01-08,1.68,1.73,0.05,59,-0.05,-2.95,94.78,NaN,NaN
5,iag,2024-01-09,1.69,1.68,-0.01,59,0.01,0.59,95.37,NaN,NaN
6,iag,2024-01-10,1.66,1.69,0.03,60,-0.03,-1.80,93.57,NaN,NaN
7,iag,2024-01-11,1.66,1.66,0.00,60,0.00,0.00,93.57,NaN,100.00
8,iag,2024-01-12,1.68,1.66,-0.02,59,0.02,1.18,94.75,NaN,101.18
9,iag,2024-01-15,1.68,1.68,0.00,59,0.00,0.00,94.75,NaN,101.18


In [16]:
df.to_csv('iag.csv',index=False)